In [ ]:
# Import the requests library.
import requests
import numpy as np
import pandas as pd
# Import the requests library.
import requests
import datetime
from datetime import datetime
import matplotlib.pyplot as plt
# Dependencies for the wine API
import urllib
import json
# Import the API key.
from config import weather_api_key
import calendar

Token = 'tylRGHbxqiZUDNymMowCBFCuEqeULiWk'
Token_NOAA=Token
API_Token='Token d8f94b93417333267ca0edcca7dc14a9e035e1bb'

In [2]:
# https://github.com/crvaden/NOAA_API_v2
# https://towardsdatascience.com/getting-weather-data-in-3-easy-steps-8dc10cc5c859
# https://cran.r-project.org/web/packages/rnoaa/rnoaa.pdf
# file:///C:/Users/15124/Downloads/GHCND_documentation.pdf
# https://www1.ncdc.noaa.gov/pub/data/ghcn/daily/ghcnd-stations.txt

# For Loop with several zip codes


In [44]:
# Create empty lists to append the data into
date_temp_MAX=[]
value_temp_MAX=[]
df=pd.DataFrame()

zip_codes=[99336,97218]
names=['avgTemp_CA', 'avgTemp_WA']
# names=['avgTemp_WW']
# zip_codes=[97218]


df_temp_combined=pd.DataFrame()
bigdf=pd.DataFrame()


for i in range(len(zip_codes)):
    zip_=zip_codes[i]
    for year, name in zip(range(2010, 2011), names):
        year = str(year)
        #Print out 'working on year' to idenfity if script is running correctly
        print('working on year '+year)

        #make the api call for temp and precipitation
        r = requests.get(f'https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&locationid=ZIP:{zip_}&datatypeid=TMAX&datatypeid=TMIN&units=standard&limit=1000&startdate='+year+'-01-01&enddate='+year+'-12-31', headers={'token':Token})

        #Load JSON data
        d = json.loads(r.text)

        #Sort JSON data by TMAX
        sorted_json= [item for item in d['results'] if item['datatype']=='TMAX']

        #Convert list into DF
        date_temp_MAX += [item['date'] for item in sorted_json]
        date_df=pd.DataFrame(date_temp_MAX)
        date_df=date_df.apply(pd.to_datetime)
#         date_df = date_df.set_index('date').resample('M').mean()
        
        #get the actual average temperature from all average temperature readings
        value_temp_MAX += [item['value'] for item in sorted_json]
        value_df=pd.DataFrame(value_temp_MAX)
#         value_df = value_df.set_index('Date').resample('M').mean()

        
        df['Date']=date_df[0]
        df[name]=value_df[0]
        bigdf['Date']=df['Date']
        bigdf[name]=df[name]
                          
bigdf

working on year 2010
working on year 2010


,Date,avgTemp_CA
0,2010-01-01,31.0
1,2010-01-02,49.0
2,2010-01-03,50.0
3,2010-01-04,37.0
4,2010-01-05,37.0
...,...,...
360,2010-12-27,49.0
361,2010-12-28,50.0
362,2010-12-29,52.0
363,2010-12-30,44.0


In [1]:
# Import the requests library.
from numpy.lib.function_base import append
import requests
import numpy as np
import pandas as pd
import datetime
from datetime import datetime
# Dependencies for the wine API
import urllib
import json
import calendar

Token = 'tylRGHbxqiZUDNymMowCBFCuEqeULiWk'
Token_NOAA=Token
API_Token='Token d8f94b93417333267ca0edcca7dc14a9e035e1bb'
 
#%%
bigdf = pd.DataFrame()
states = { 
    'CA': {
        'zip': '97218',
        'years': ['2010', '2011']
        },
    'Wash': {
        'zip': '99336',
        'years': ['2010', '2011']
        }
    #etc...
}
for state in states:
    small_df = pd.DataFrame()
    for year in states[state]['years']:
        zip = states[state]['zip']
        df_test = pd.DataFrame()
        #Print out 'working on year' to idenfity if script is running correctly
        print('working on year '+str(year))
        #make the api call for temp and precipitation
        r = requests.get(f'https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&locationid=ZIP:{zip}&datatypeid=TMAX&datatypeid=TMIN&units=standard&limit=1000&startdate='+year+'-01-01&enddate='+year+'-12-31', headers={'token':Token})
        #Load JSON data
        d = json.loads(r.text)
        #     print(d)
        #e = json.loads(p.text)
        items_MAX = [item for item in d['results'] if item['datatype']=='TMAX']
        items_MIN = [item for item in d['results'] if item['datatype']=='TMIN']

        # #get the date field from all average temperature readings
        list_max_temp = [item['date'].split('T')[0] for item in items_MAX]
        list_min_temp = [item['date'].split('T')[0] for item in items_MIN]

        df_test['date'] = list_max_temp
        df_test['avgMaxTemp'+state] = np.nan
        for item in items_MAX:
            date = item['date'].split('T')[0]
            df_test.loc[df_test['date'] == date, ['avgMaxTemp'+state]] = item['value']

        df_test['avgMinTemp'+state] = np.nan
        for item in items_MIN:
            date = item['date'].split('T')[0]
            df_test.loc[df_test['date'] == date, ['avgMinTemp'+state]] = item['value']
        small_df = pd.concat([small_df,df_test])
    if bigdf.empty:
        bigdf = small_df
    else:
        bigdf = pd.merge(bigdf,small_df)
bigdf


working on year 2010
working on year 2011
working on year 2010
working on year 2011


,date,avgMaxTempCA,avgMinTempCA,avgMaxTempWash,avgMinTempWash
0,2010-01-01,53.0,40.0,31.0,22.0
1,2010-01-02,53.0,41.0,49.0,27.0
2,2010-01-03,46.0,40.0,50.0,31.0
3,2010-01-04,45.0,41.0,37.0,31.0
4,2010-01-05,47.0,42.0,37.0,34.0
...,...,...,...,...,...
725,2011-12-27,53.0,41.0,49.0,36.0
726,2011-12-28,57.0,49.0,58.0,42.0
727,2011-12-29,54.0,46.0,62.0,42.0
728,2011-12-30,55.0,37.0,53.0,44.0


In [ ]:
# Create empty lists to append the data into
dates_temp = []
temps = []
dates_prcp = []
prcp = []
temps_MAX=[]
temps_MIN=[]
dates_temp_MAX=[]
dates_temp_MIN=[]
# zip_codes=[97218,99336]
zip_codes=[97218]

names=['avgTemp_WA']
# names=['avgTemp_WW']
 # #initialize dataframe
df_temp_MAX = pd.DataFrame()
df_temp_MIN = pd.DataFrame()
df_temp_AVG = pd.DataFrame()
df_temp_combined=pd.DataFrame()
bigdf=pd.DataFrame()

for i,name in zip(range(len(zip_codes)), names):
    zip_=zip_codes[i]
    #Identify what years we want Temp and Prcp data for
    for year in range(2010, 2011):
        year = str(year)
        #Print out 'working on year' to idenfity if script is running correctly
        print('working on year '+year)
        #make the api call for temp and precipitation
        r = requests.get(f'https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&locationid=ZIP:{zip_}&datatypeid=TMAX&datatypeid=TMIN&units=standard&limit=1000&startdate='+year+'-01-01&enddate='+year+'-12-31', headers={'token':Token})
        
        #Load JSON data
        d = json.loads(r.text)
        print(d)
        df = pd.DataFrame.from_dict(d, orient='columns')

In [ ]:
bigdf['california']=1
bigdf['washington']=1
bigdf['oregon']=1
bigdf

In [ ]:
timestamp='2010-01-31T00:00:00'
timestamps=timestamp.split('T')[0]
print(bigdf[timestamps])

In [ ]:
dates_temp_MAX

In [ ]:
df_temp_MIN['avgTemp']

In [ ]:
#Create empty lists to append the data into
dates_temp = []
temps = []
dates_prcp = []
prcp = []
zip_codes=[97218,99336]
names=['avgTemp_WW', 'avgTemp_CA']

dates_temp_MAX=[]
dates_temp_MIN=[]
temps_MAX=[]
temps_MIN=[]

 # #initialize dataframe
df_temp_MAX = pd.DataFrame()
df_temp_MIN = pd.DataFrame()
df_temp_AVG = pd.DataFrame()
df_temp_combined=pd.DataFrame()
bigdf=pd.DataFrame()


for i,name in zip(range(len(zip_codes)), names):
    zip_=zip_codes[i]
    for year in range(2010, 2011):
        year = str(year)
        #Print out 'working on year' to idenfity if script is running correctly
        print('working on year '+year)

        #make the api call for temp and precipitation
        r = requests.get(f'https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&locationid=ZIP:{zip_}&datatypeid=TMAX&datatypeid=TMIN&units=standard&limit=1000&startdate='+year+'-01-01&enddate='+year+'-12-31', headers={'token':Token})

        #Load JSON data
        d = json.loads(r.text)
        #print(d)

        #Get TAVG and PRCP data
        avg_temps_MAX = [item for item in d['results'] if item['datatype']=='TMAX']
        avg_temps_MIN = [item for item in d['results'] if item['datatype']=='TMIN']

        #get the date field from all average temperature readings
        dates_temp_MAX += [item['date'] for item in avg_temps_MAX]
        dates_temp_MIN += [item['date'] for item in avg_temps_MIN]

        #get the actual average temperature from all average temperature readings
        temps_MAX += [item['value'] for item in avg_temps_MAX]
        temps_MIN += [item['value'] for item in avg_temps_MIN]

        #populate date and average temperature fields (cast string date to datetime
#         df_temp_MAX['date'] = [datetime.strptime(d, "%Y-%m-%dT%H:%M:%S") for d in dates_temp_MAX]
#         df_temp_MIN['date'] = [datetime.strptime(d, "%Y-%m-%dT%H:%M:%S") for d in dates_temp_MIN]

        #convert to degrees F
        df_temp_MAX['avgTemp'] = temps_MAX
        df_temp_MIN['avgTemp'] = temps_MIN
        #df_temp_MAX = df_temp_MAX.set_index('date').resample('M').mean()
        #df_temp_MIN = df_temp_MIN.set_index('date').resample('M').mean()

        df_temp_MIN.reset_index(inplace=True)
        df_temp_MIN.rename(columns={'avgTemp':name}, inplace=True)
        df_temp_combined=df_temp_MIN[['date', name]]
#         df_temp_combined['date']=pd.datetime(df_temp_combined['date'], errors='coerce')
        bigdf['date']=df_temp_combined['date']
        bigdf[name]=df_temp_combined[name]
# bigdf

In [ ]:
df_temp_MIN